# Initialization

## Formatter

In [1]:
using JupyterFormatter
enable_autoformat()

1-element Vector{Function}:
 format_current_cell (generic function with 1 method)

## Load Packages

In [36]:
using HalfIntegers
using LinearAlgebra
using ElasticArrays
using JLD2
using Distributed
using DelimitedFiles
using Random
using CSV
using DataFrames
using LsqFit

## Plot style

In [37]:
using Plots
using Plots.PlotMeasures
using LaTeXStrings

# set backend and style
pgfplotsx()
theme(:default)
default(
    markerstrokewidth=0,
    markerstrokealpha=0,
    linewidth=2,
    #grid=:none,   
    gridlinewidth=0.5,
    markersize=5,
    markershape=:circle,
    tickfontsize=18,
    size=(900, 600),
    legendfontsize=18,
    guidefontsize=20,
    titlefontsize=20,
    legend=(0.03, 0.98),
    xticks=0:10,
    foreground_color_axis="black",
    foreground_color_border="black",
    foreground_color_guide="darkorange",
    foreground_color_text="black",
    guidefontcolor="black",
    plot_titlefontcolor="black",
    titlefontcolor="black",
    shape=[:circle],
)

## Data

In [38]:
DATA_FOLDER = "../../data";
STORE_FOLDER = "./Plots";

alpha_collection = [3, 4, 5, 6]
alpha_range = length(alpha_collection)
immirzi_collection = [1, 5]
immirzi_range = length(immirzi_collection)
T_sampling_parameter = 100
Dl_min = 0
Dl_max = 10
Dl_range = Dl_max - Dl_min + 1
angular_spins = [[1], [1.5], [2], [2.5], [3], [3.5], [4], [4.5], [5]]
angular_spins_range = length(angular_spins);

## Useful Function 

In [39]:
function Fromj0ToMass(j0_float, immirzi)
    round(sqrt(2 * j0_float * immirzi), digits=2)
end

Fromj0ToMass (generic function with 2 methods)

In [40]:
function Fromj0Tojpm(j0_float)
    round(Int64, 2 * (j0_float / (sqrt(6)))) / 2
end

Fromj0Tojpm (generic function with 1 method)

# Computational times

In [61]:
workers = 64
threads = 1

comp_times = Matrix(
    DataFrame(
        CSV.File(
            "../../immirzi_$(immirzi)_workers_$(workers)_threads_$(threads).csv",
            header=true,
        ),
    ),
);

In [18]:
pl1 = plot(
    1:11,
    [
        comp_times[:, 1],
        comp_times[:, 2],
        comp_times[:, 3],
        comp_times[:, 4],
        comp_times[:, 5],
        comp_times[:, 6],
        comp_times[:, 7],
        comp_times[:, 8],
        comp_times[:, 9],
    ],
    label=[L"j_0=1,\,j_{\pm}=0.5" L"j_0=1.5,\,j_{\pm}=0.5" L"j_0=2,\,j_{\pm}=1" L"j_0=2.5,\,j_{\pm}=1" L"j_0=3,\,j_{\pm}=1" L"j_0=3.5,\,j_{\pm}=1.5" L"j_0=4,\,j_{\pm}=1.5" L"j_0=4.5,\,j_{\pm}=2" L"j_0=5,\,j_{\pm}=2"],
    markershape=:circle,
    yaxis=:log,
    xaxis=:log,
    legend=(0.10, 0.97),
    legendfontsize=18,
)
xticks!(
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    string.([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]),
)
xlabel!(L"\Delta l")
ylabel!(L"\textrm{seconds}")
title!(L"\textrm{Computation time of EPRL vertices}")
savefig(
    "./plots/narval_computation_workers_$(workers)_threads_$(threads)_immirzi_$(immirzi).pdf",
)

"/home/frisus/Scrivania/BlackToWhiteHole/notebooks/Julia/plots/narval_computation_workers_64_threads_1_immirzi_5.pdf"

# Amplitude

In [41]:
function AmplitudeAbsSquaredLoad(
    DATA_FOLDER,
    j0,
    jpm,
    alpha,
    immirzi,
    T,
    Dl_min=0,
    Dl_max=10,
    K0=0.5,
    Kpm=0.5)

    if j0 > 5
        K0 = 0.0
        Kpm = 0.0
    end

    Matrix(
        DataFrame(
            CSV.File(
                "$(DATA_FOLDER)/amplitude_data/j0=$(j0)_jpm=$(jpm)/K0_$(K0)_Kpm_$(Kpm)/immirzi_$(immirzi)/alpha_$(alpha)/Dl_range_$(Dl_range)/amplitude_abs_sq_T_$(T).csv",
                header=true,
            ),
        ),
    )

end

function AmplitudeRescaling(ampls)
    rescaling_factor = 1000000000000000000000000000000000000000000000000  # 10^50
    ampls[:] .= ampls[:] .* rescaling_factor
end

function TRange(m, immirzi, T_sampling_parameter)
    LinRange(0, 4 * pi * m / immirzi, T_sampling_parameter)
end

TRange (generic function with 1 method)

In [42]:
ampls_path = "$(STORE_FOLDER)/ampls/T_sampling_parameter_$(T_sampling_parameter)"
mkpath("$(ampls_path)")

ampls_collection = Array{Float64,5}(undef, T_sampling_parameter, Dl_range, angular_spins_range, alpha_range, immirzi_range)
Lifetime_x_coordinates = Float64[]

for angular_spin_index in eachindex(angular_spins)

    user_conf = angular_spins[angular_spin_index]
    j0_float = user_conf[1]
    j0 = half(2 * j0_float)

    if length(user_conf) == 1
        jpm_float = round(Int64, 2 * (j0_float / (sqrt(6)))) / 2
        jpm = half(2 * jpm_float)
    else
        jpm_float = user_conf[2]
        jpm = half(2 * jpm_float)
    end

    for immirzi_index in eachindex(immirzi_collection)

        immirzi = immirzi_collection[immirzi_index]
        m = Fromj0ToMass(j0, immirzi)
        T_range = TRange(m, immirzi, T_sampling_parameter)
        push!(Lifetime_x_coordinates, m)

        DeltaT = Int(T_sampling_parameter / 5)
        X_ticks = [0:DeltaT:T_sampling_parameter;]
        X_labels = String[]

        for x in X_ticks[1:end-1]
            l = T_range[x+1]
            l = round(l, digits=2)
            push!(X_labels, "$(l)")
        end

        final_x_tick = T_range[end]
        push!(X_labels, L"\frac{4 \pi m}{\gamma}")

        ampls_path_immirzi = "$(ampls_path)/immirzi_$(immirzi)"

        for alpha_index in eachindex(alpha_collection)

            alpha = alpha_collection[alpha_index]
            ampls = AmplitudeAbsSquaredLoad(DATA_FOLDER, j0_float, jpm_float, alpha, immirzi, T_sampling_parameter, Dl_min, Dl_max)
            AmplitudeRescaling(ampls)
            ampls_collection[:, :, angular_spin_index, alpha_index, immirzi_index] .= ampls[:, :]

            ampls_path_immirzi_alpha = "$(ampls_path_immirzi)/alpha_$(alpha)"
            mkpath(ampls_path_immirzi_alpha)

            pl1 = plot(
                1:T_sampling_parameter,
                [ampls[:, 1] ampls[:, 2] ampls[:, 3] ampls[:, 4] ampls[:, 5] ampls[:, 6] ampls[:, 7] ampls[:, 8] ampls[:, 9] ampls[:, 10] ampls[:, 11]],
                label=["Δl=0" "Δl=1" "Δl=2" "Δl=3" "Δl=4" "Δl=5" "Δl=6" "Δl=7" "Δl=8" "Δl=9" "Δl=10"],
                legend=(0.20, 0.90),
                #label = "",
                markershape=:circle,
                grid=:true,
                xlims=(0, T_sampling_parameter + 1),
                xticks=(X_ticks, X_labels),
                palette=palette([:skyblue, :purple], Dl_range),
                legendfontsize=20)
            xlabel!(L"T")
            title!(L"|W_{\alpha=%$alpha}  \, \left(m=%$m, \, T \right) |^2 \times 10^{50}")
            savefig("$(ampls_path_immirzi_alpha)/j0_$(j0_float).pdf")

        end

    end

end

# Lifetime

In [29]:
#Y_ticks = [exp(-60):1.0;]
#Y_labels = [L"e^{-60}",L"e^{-30}",L"1"]

pl1 = plot(
    Lifetime_x_coordinates,
    [ampls_collection[end, 1, :] ampls_collection[end, 2, :] ampls_collection[end, 3, :] ampls_collection[end, 4, :] ampls_collection[end, 5, :] ampls_collection[end, 6, :] ampls_collection[end, 7, :] ampls_collection[end, 8, :] ampls_collection[end, 9, :]],# ampls_collection[100, 10, :] ampls_collection[100, 11, :]],
    label=["Δl=0" "Δl=1" "Δl=2" "Δl=3" "Δl=4" "Δl=5" "Δl=6" "Δl=7" "Δl=8"], #"Δl=9" "Δl=10"],
    legend=(0.10, 0.60),
    #label = "",
    markershape=:circle,
    grid=:true,
    #xscale = :log,
    yscale=:ln,
    #xlims=(0, T_sampling_parameter + 1),
    #xticks=(X_ticks, X_labels),
    #yticks=(Y_ticks, Y_labels),
    palette=palette([:skyblue, :purple], 9),
    legendfontsize=20)
xlabel!(L"m")
title!(L"\mu \left( m, \, T \right) |W_{\alpha}  \, \left(m, \, T \right) |^2")
savefig("$(ampls_path)/Lifetime_alpha_$(alpha)_Immirzi_$(immirzi).pdf")

"/home/frisus/Scrivania/BlackToWhiteHole/notebooks/Julia/plots/ampls/T_sampling_parameter_100/Lifetime_alpha_3_Immirzi_5.pdf"

# Crossing Time

In [43]:
function CrossingTimeLoad(DATA_FOLDER, j0, jpm, alpha, immirzi, T, Dl_range, K0=0.5, Kpm=0.5)

    Matrix(
        DataFrame(
            CSV.File(
                "$(DATA_FOLDER)/amplitude_data/j0=$(j0)_jpm=$(jpm)/K0_$(K0)_Kpm_$(Kpm)/immirzi_$(immirzi)/alpha_$(alpha)/Dl_range_$(Dl_range)/crossing_time_$(T).csv",
                header=true,
            ),
        ),
    )

end

CrossingTimeLoad (generic function with 3 methods)

In [44]:
crossing_times_collection = Array{Float64,3}(undef, angular_spins_range, alpha_range, immirzi_range)
mass_values_collection = Array{Float64,2}(undef, angular_spins_range, immirzi_range)
mass_values_string_collection = Array{String,2}(undef, angular_spins_range, immirzi_range)
semiclassical_estimate_collection = Array{Float64,2}(undef, angular_spins_range, immirzi_range)

for angular_spin_index in eachindex(angular_spins)

    user_conf = angular_spins[angular_spin_index]
    j0_float = user_conf[1]
    j0 = half(2 * j0_float)

    if length(user_conf) == 1
        jpm_float = round(Int64, 2 * (j0_float / (sqrt(6)))) / 2
        jpm = half(2 * jpm_float)
    else
        jpm_float = user_conf[2]
        jpm = half(2 * jpm_float)
    end

    for immirzi_index in eachindex(immirzi_collection)

        immirzi = immirzi_collection[immirzi_index]
        m = Fromj0ToMass(j0, immirzi)
        mass_values_collection[angular_spin_index, immirzi_index] = m
        mass_values_string_collection[angular_spin_index, immirzi_index] = "$(m)"
        semiclassical_estimate_collection[angular_spin_index, immirzi_index] = 2 * m * pi / immirzi

        for alpha_index in eachindex(alpha_collection)

            alpha = alpha_collection[alpha_index]
            crossing_time = CrossingTimeLoad(DATA_FOLDER, j0_float, jpm_float, alpha, immirzi, T_sampling_parameter, Dl_range)
            crossing_times_collection[angular_spin_index, alpha_index, immirzi_index] = crossing_time[end] # only last value of truncation parameter

        end

    end

end

In [46]:
crossing_times_path = "$(STORE_FOLDER)/crossing_times/T_sampling_parameter_$(T_sampling_parameter)"

for immirzi_index in eachindex(immirzi_collection)

    immirzi = immirzi_collection[immirzi_index]
    crossing_times_path_immirzi = "$(crossing_times_path)/immirzi_$(immirzi)"
    mkpath("$(crossing_times_path_immirzi)")

    pl1 = scatter(
        mass_values_collection[2:end, immirzi_index],
        [[crossing_times_collection[2:end, 1, immirzi_index] crossing_times_collection[2:end, 2, immirzi_index] crossing_times_collection[2:end, 3, immirzi_index] crossing_times_collection[2:end, 4, immirzi_index]]],
        label=[L"\alpha=3" L"\alpha=4" L"\alpha=5" L"\alpha=6"],
        legend=(0.10, 0.97),
        markersize=6,
        linewidth=-1,
        markershape=:circle,
        #linestyle = :none,
        legendfontsize=18,
        xticks=mass_values_collection[2:end, immirzi_index])
    xlabel!(L"m")
    title!(L"\tau \left(m \right)")

    plot!(
        mass_values_collection[2:end, immirzi_index],
        [[semiclassical_estimate_collection[2:end, immirzi_index]]],
        label="semicl.",
        linestyle=:dash,
        markershape=:none)

    savefig("$(crossing_times_path_immirzi)/Tc_immirzi_$(immirzi).pdf")

end